# Creating an Unilateral Closing Transaction

In this section we'll broadcast an unilateral closing transaction created on the previous notebook. We'll test it using bitcoin core in regtest mode.

## Prerequisite knowledge
### For all notebooks
- A high level understanding of the bitcoin. e.g. [Mastering Bitcoin](https://github.com/bitcoinbook/bitcoinbook), in particular [Chapter 6](https://github.com/bitcoinbook/bitcoinbook/blob/develop/ch06.asciidoc).
- A conceptual understanding of [hash functions](https://www.thesslstore.com/blog/what-is-a-hash-function-in-cryptography-a-beginners-guide).
- [Hexadecimal notation](https://inst.eecs.berkeley.edu/~cs61bl/r//cur/bits/decimal-binary-hex.html?topic=lab28.topic&step=2&course=) and [endianness](https://www.freecodecamp.org/news/what-is-endianness-big-endian-vs-little-endian/).
- A high level understanding of the lightning e.g. [Mastering Lightning Network](https://github.com/lnbook/lnbook), in particular [Chapter7](https://github.com/lnbook/lnbook/blob/develop/07_payment_channels.asciidoc), [Chapter 8](https://github.com/lnbook/lnbook/blob/develop/08_routing_htlcs.asciidoc) and [Chapter 9](https://github.com/lnbook/lnbook/blob/develop/09_channel_operation.asciidoc).

### Specific to this notebook:
- SHA256, HASH256, HASH160 - '[Hash Functions chapter](https://github.com/MPins/lightning-tx-tutorial/blob/main/appendix/hash-functions.ipynb)'
- Bech32 addresses - '[Addresses chapter](https://github.com/MPins/lightning-tx-tutorial/blob/main/appendix/Addresses.ipynb)'
- Bitcoin Script basics - '[Bitcoin Script chapter](https://github.com/MPins/lightning-tx-tutorial/blob/main/appendix/Bitcoin%20Script.ipynb)'
- Lightning Network BOLT #2: '[Peer Protocol for Channel Management](https://github.com/lightning/bolts/blob/master/02-peer-protocol.md#channel-establishment-v1)'
- Lightning Network Bolt #3: '[Funding Transaction Output](https://github.com/lightning/bolts/blob/master/03-transactions.md#funding-transaction-output)'

## What is a Unilateral Closing Transaction?

A Unilateral Closing Transaction, in the Lightning Network, refers to force close, where one party decides to close the channel without the agreement or cooperation of the other party. This can happen when one party is unable to contact the other or if there's a dispute about the channel balance.


## Setup 
### Requirements
For this chapter we'll need the Bitcoin Core. This notebook has been tested with [v28.0](https://github.com/bitcoin/bitcoin/releases/tag/v28.0).

Below, set the paths for:
1. The bitcoin core functional test framework directory.
2. The directory containing lightning-tx-tutorial.

**You'll need to edit these next two lines for your local setup.**

In [1]:
# run notebook
%run "/home/pins-dev/Projects/lightning-tx-tutorial/Chapter 2 - Commitment Transactions/Commitment Transactions with HTLC in Transit.ipynb"

Alice Per Commitment Seed 34b581ec20bf2c6cae3d4d4dcbfddc8a3727a1e9a57c55f3520e770607898c06
Bob Per Commitment Seed 89c994b3ddad4698acee71e42d8bcace48eea739caaba371eb110e77663ec56d
Alice Revocation Basepoint Private Key: c17ac3952ca414190074d1e59ea03fbae253196173908dc8b131af6bd2cc8161
Alice Revocation Basepoint Public Key: 03649c4f865bec74b0a186deef4defad51cfdc141443e38074ea05a7835a953a49
Alice HTLC Basepoint Private Key: 763ae49a20e6668c88602c782716dd83ba6c4cc0333b38810e2bcd7b22c871ac
Alice HTLC Basepoint Public Key: 02816fde4150e4dfcac94eff0b821448fb70f57a56148ba2206cd9b2fd0cc20bdf
Alice Payment Basepoint Private Key: 72d8c12971b58076a1f27eb7938ca442f0b210762b23637443ac2e99dac352a6
Alice Payment Basepoint Public Key: 025f892a06124391e2f38ce35d943cdc09f63e203330dbd9cb6113a903e0738458
Alice Delayed Payment Basepoint Private Key: 7cafce00c54e7241894dcc7c3beaca29dd354139fdb6182198d6c5f1063bfe8d
Alice Delayed Payment Basepoint Public Key: 034aa35219136bb238e072341b20a4bf8fb44a83cdb73dd2bd9

## The Unilateral Closing Transaction

A unilateral close happens when one party unilaterally publishes their latest commitment transaction to the Bitcoin blockchain, effectively force-closing the channel.

Reasons this might happen:
- your channel partner is offline and cannot be contacted to initiate a mutual close.
- your channel partner is online, but is not responding to requests to initiate a mutual close.
- your channel partner is online and your nodes are negotiating a mutual close, but they become stuck and cannot reach a resolution.

Let's imagine that Alice want to force close the channel with Bob, so she needs just publish the last commitment transaction.

In [2]:
alice_signed_commitment_txid = node.sendrawtransaction(alice_signed_commitment_tx.hex())
# Mine a block to confirm the tx
mining_address = node.getnewaddress()
txid = node.generatetoaddress(nblocks=1, address=mining_address, invalid_call=False)
height = node.getblockcount()
print("Current block height:", height)
print("Force Close txid", alice_signed_commitment_txid)

Current block height: 104
Force Close txid 7a8baa4591e4d726a82bae863963278b440fee5f146e3d6d673ec652412d256b


The channel was forced close by Alice.